In [1]:
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt
import random
import math

In [2]:
def save_img(path, img):
    cv2.imwrite(path, img)
    print(path, "is saved!")


def display_img(img):
    cv2.imshow('Result', img)
    cv2.waitKey()


def read_img(path):
    image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    # if need double type, uncomment the following
    # out = image.astype(float)
    return image


def read_colorimg(path):
    image = cv2.imread(path)
    # if need double type, uncomment the following
    # out = image.astype(float)
    return image

In [11]:
def homography_transform(X, H):
    # TODO
    # Perform homography transformation on a set of points X
    # using homography matrix H
    # Input - a set of 2D points in an array with size (N,2)
    #         a 3*3 homography matrix 
    # Output - a set of 2D points in an array with size (N,2)

    Y = None
    return Y


def fit_homography(XY):
    # TODO
    # Given two set of points X, Y in one array,
    # fit a homography matrix from X to Y
    # Input - an array with size(N,4), each row contains two
    #         points in the form[x^T_i,y^T_i]1×4
    # Output - a 3*3 homography matrix
    H = None
    return H


def p1():
    # 1.2.3 - 1.2.5
    # TODO
    # 1. load points X from p1/transform.npy
    X = np.load("p1/transform.npy")
    print(X)

    # 2. fit a transformation y=Sx+t

    # 3. transform the points 

    # 4. plot the original points and transformed points

    """
    # 1.2.6 - 1.2.8
    case = 8 # you will encounter 8 different transformations
    for i in range(case):
        XY = np.load('p1/points_case_'+str(i)+'.npy')
        # 1. generate your Homography matrix H using X and Y
        #
        #    specifically: fill function fit_homography() 
        #    such that H = fit_homography(XY)
        H = fit_homography(XY)
        # 2. Report H in your report
        print(H)
        # 3. Transform the points using H
        #
        #    specifically: fill function homography_transform
        #    such that Y_H = homography_transform(X, H)
        Y_H = homography_transform(XY[:,:2], H)
        # 4. Visualize points as three images in one figure
        # the following codes plot figure for you
        plt.scatter(XY[:,1],XY[:,0],c="red") #X
        plt.scatter(XY[:,3],XY[:,2],c="green") #Y
        plt.scatter(Y_H[:,1],Y_H[:,0],c="blue") #Y_hat
        plt.savefig('./case_'+str(i))
        plt.close()
    """

In [7]:
def stitchimage(imgleft, imgright):
    # TODO
    # 1. extract descriptors from images
    #    you may use SIFT/SURF of opencv

    # 2. select paired descriptors

    # 3. run RANSAC to find a transformation
    #    matrix which has most innerliers

    # 4. warp one image by your transformation 
    #    matrix
    #
    #    Hint: 
    #    a. you can use opencv to warp image
    #    b. Be careful about final image size

    # 5. combine two images, use average of them
    #    in the overlap area

    pass


def p2(p1, p2, savename):
    # read left and right images
    imgleft = read_colorimg(p1)
    imgright = read_colorimg(p2)
    # stitch image
    output = stitchimage(imgleft, imgright)
    # save stitched image
    save_img('./' + savename + '.jpg', output)

In [12]:
# Problem 1
p1()

# Problem 2
# p2('p2/uttower_left.jpg', 'p2/uttower_right.jpg', 'uttower')
# p2('p2/bbb_left.jpg', 'p2/bbb_right.jpg', 'bbb')

# Problem 3
# TODO
# add your code for implementing Problem 3
# 
# Hint:
# you can use functions in Problem 2 here

[[ -4.55198471  -2.76449494 -10.01344842   7.52905765]
 [  0.93256876   1.68766484  -1.94371611  -4.65875269]
 [  0.98430732   4.12950979  -4.67581318  -2.32232542]
 [ -1.27448748  -1.4299302   -2.58751662  -2.55743011]
 [  3.46042361  -4.4574087   10.65883589 -16.91429949]
 [  0.51460732  -2.88654683   1.61603561  -5.81666795]
 [ -2.61029107  -1.02552495  -3.88573887   4.61747111]
 [ -0.64540029  -4.57245754  -0.33699174  -5.57592803]
 [  4.57338298  -1.30622453   6.64874117 -19.58508971]
 [ -2.28154437   1.77425485  -7.79063009   5.93667841]
 [  2.57998375   4.06980282  -2.13326858  -8.62995405]
 [ -1.55113263   2.84844336  -9.31672319   3.66316343]
 [ -4.67958662   3.44390328 -14.43110905  14.64330534]
 [  2.44068102  -2.49875629   5.85457985 -12.81696339]
 [ -1.24760852  -4.87391209   1.22891971  -5.46343435]
 [  3.11563037   1.96489228   2.87911893 -10.58987806]
 [  2.54895101   1.62529504   0.93720938 -11.38715834]
 [ -3.51755547  -2.93189971  -6.59295875   3.47797929]
 [  4.4845